In [58]:
import random


# Data structure for courses which also includes lab courses and all that
courses = {
    "CS101": {"name": "Programming Fundamentals", "type": "theory"},
    "CS102": {"name": "Data Structures and Algorithms", "type": "theory"},
    "CS103": {"name": "Database Management System", "type": "theory"},
    "CS104": {"name": "Computer Networks", "type": "theory"},
    "CS105": {"name": "Operating Systems", "type": "theory"},
    "CS106": {"name": "Software Engineering", "type": "theory"},
    "CS107": {"name": "Artificial Intelligence", "type": "theory"},
    "CS108": {"name": "Theory Of Automata", "type": "theory"},
    "CS109": {"name": "Algorithms", "type": "theory"},
    "CS110": {"name": "Web Programming", "type": "theory"},
    "CS111": {"name": "Object Oriented Programming", "type": "theory"},
    "CS201": {"name": "Programming Fundamentals Lab", "type": "lab"},
    "CS202": {"name": "Data Structures Lab", "type": "lab"},
    "CS203": {"name": "Database Lab", "type": "lab"},
    "CS204": {"name": "Computer Networks Lab", "type": "lab"},
    "CS205": {"name": "Operating Systems Lab", "type": "lab"},
    "CS206": {"name": "Artificial Intelligence Lab", "type": "lab"},
    
}

# Data structure for professors
professors = {
    "Prof1": {"name": "Shams Farooq", "courses": ["CS101", "CS102", "CS103", "CS104"]},
    "Prof2": {"name": "Usama Imtiaz", "courses": ["CS105", "CS106", "CS107"]},
    "Prof3": {"name": "Hassan Mujtaba", "courses": ["CS108", "CS109", "CS110", "CS111"]},
    "Prof4": {"name": "Laraib Asif", "courses": ["CS201", "CS202", "CS203"]},
    "Prof5": {"name": "Eman Mirza", "courses": ["CS204", "CS205", "CS206"]},
    
}

# Data structure for sections
sections = {
    "CS-A": {"students": [], "courses": ["CS101", "CS102", "CS103"], "batch": 20, "department": "CS"},
    "DS-B": {"students": [], "courses": ["CS104", "CS105", "CS106"], "batch": 21, "department": "DS"},
    "CY-C": {"students": [], "courses": ["CS107", "CS108", "CS109"], "batch": 22, "department": "CY"},
    "AI-D": {"students": [], "courses": ["CS110", "CS111"], "batch": 23, "department": "AI"},
    
}

# Data structure for rooms
rooms = {
    "C301": {"capacity": 60, "type": "classroom"},
    "C302": {"capacity": 60, "type": "classroom"},
    "C303": {"capacity": 60, "type": "classroom"},
    "C304": {"capacity": 60, "type": "classroom"},
    "C305": {"capacity": 60, "type": "classroom"},
    "Hall1": {"capacity": 120, "type": "large_hall"},
    "Hall2": {"capacity": 120, "type": "large_hall"},
    
}


# Example timetable slots for Monday this is just an exmaple and it is not being used in the code
timetable = {
    "Monday": {
        "08:30": {"room": "C301", "course": "CS101", "professor": "Prof1", "section": "CS-A", "batch": 20, "department": "CS", "type": "theory"},
        "10:00": {"room": "C302", "course": "CS102", "professor": "Prof1", "section": "CS-A", "batch": 20, "department": "CS", "type": "theory"},
        "11:30": {"room": "C303", "course": "CS103", "professor": "Prof1", "section": "CS-A", "batch": 20, "department": "CS", "type": "theory"},
        "13:00": {"room": "C304", "course": "CS104", "professor": "Prof2", "section": "DS-B", "batch": 21, "department": "DS", "type": "theory"},
        "14:30": {"room": "C305", "course": "CS105", "professor": "Prof2", "section": "DS-B", "batch": 21, "department": "DS", "type": "theory"},
        
    },
    
}


# Function to encode a timetable into a chromosome
def encode_tt(timetable):
    chromosome = ""
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"] # with days of the week except saturday and sunday
    time_slots = ["08:30", "10:00", "11:30", "13:00", "14:30"] # time slots which are necessary
    
    for day in days:
        for time_slot in time_slots:
            slot_info = timetable[day][time_slot]
            if slot_info:
                # Encode course, professor, room, and section
                course_code = bin(int(slot_info["course"][2:]) - 101)[2:].zfill(5)
                professor_code = bin(int(slot_info["professor"][4:]) - 1)[2:].zfill(3)
                room_code = bin(int(slot_info["room"][1:]) - 301)[2:].zfill(3)
                section_code = bin(ord(slot_info["section"][3:]) - 65)[2:].zfill(2)
                # Append encoded information to the chromosome
                chromosome += course_code + professor_code + room_code + section_code
            else:
                # If the slot is empty, append zeros to the chromosome
                chromosome += "0" * 13
    
    return chromosome


# Function to decode a chromosome into a timetable
def decode_tt(chromosome):
    timetable = {}
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"] # with days of the week except saturday and sunday
    time_slots = ["08:30", "10:00", "11:30", "13:00", "14:30"] # time slots which are necessary
    index = 0
    
    for day in days:
        timetable[day] = {}
        for time_slot in time_slots:
            course_code = int(chromosome[index:index+5], 2)
            professor_code = int(chromosome[index+5:index+8], 2)
            room_code = int(chromosome[index+8:index+11], 2)
            section_code = chr(int(chromosome[index+11:index+13], 2) + 65)
            index += 13
            
            if course_code != 0:
                course_key = f"CS{course_code + 101}"
                professor_key = f"Prof{professor_code + 1}"
                room_key = f"C{room_code + 301}"
                section_key = f"CS-{section_code}"
                timetable[day][time_slot] = {
                    "room": room_key,
                    "course": course_key,
                    "professor": professor_key,
                    "section": section_key,
                    "batch": sections[section_key]["batch"],
                    "department": sections[section_key]["department"],
                    "type": courses[course_key]["type"]
                }
            else:
                timetable[day][time_slot] = {}
    
    return timetable


# Generate timetable function
def generate_tt():
    new_tt = {}
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

    # Initialize the timetable with empty slots for each day and time slot
    for day in days:
        new_tt[day] = {time_slot: {} for time_slot in ["08:30", "10:00", "11:30", "13:00", "14:30"]}

    # Generate timetable entries for each section and course
    for section, section_info in sections.items():
        for course in section_info["courses"]:
            slots_required = 2 if courses[course]["type"] == "lab" else 1

            # Randomize the time slots to avoid bias
            time_slots = ["08:30", "10:00", "11:30", "13:00", "14:30"]
            random.shuffle(time_slots)

            # Loop over the days in a circular manner to ensure fairness in scheduling
            for i in range(len(days)):
                day = days[(i + len(days)) % len(days)]
                
                # Reset availability dictionaries at the beginning of each day iteration
                available_rooms = {room: True for room in rooms}
                available_professors = {professor: True for professor in professors}
                available_courses = {course: True for course in courses}

                for time_slot in time_slots:
                    if new_tt[day][time_slot] == {}:
                        # Check room availability
                        available_room = None
                        for room, is_available in available_rooms.items():
                            if is_available and rooms[room]["type"] == "classroom" and section_info["batch"] <= rooms[room]["capacity"]:
                                available_room = room
                                break
                        if not available_room:
                            continue

                        # Check professor availability
                        available_professor = None
                        for professor, is_available in available_professors.items():
                            if is_available and course in professors[professor]["courses"]:
                                available_professor = professor
                                break
                        if not available_professor:
                            continue

                        # Assign room, professor, etc. to the slot
                        new_tt[day][time_slot] = {
                            "room": available_room,
                            "course": course,
                            "professor": available_professor,
                            "section": section,
                            "batch": section_info["batch"],
                            "department": section_info["department"],
                            "type": courses[course]["type"]
                        }

                        # Mark the room and professor as unavailable for subsequent slots
                        available_rooms[available_room] = False
                        available_professors[available_professor] = False
                        available_courses[course] = False  # Mark course as assigned to a slot

                        # Reduce the number of slots required
                        slots_required -= 1

                    if slots_required == 0:
                        break  # Exit the loop if all slots are assigned

                if slots_required == 0:
                    break  # Exit the outer loop if all slots are assigned

    # Ensure all courses are assigned to professors
    for course, assigned in available_courses.items():
        if assigned:
            available_professor = None
            for professor, is_available in available_professors.items():
                if is_available and course in professors[professor]["courses"]:
                    available_professor = professor
                    break
            if available_professor:
                # Find a random available time slot
                day = random.choice(days)
                time_slot = random.choice(time_slots)
                while new_tt[day][time_slot] != {}:
                    day = random.choice(days)
                    time_slot = random.choice(time_slots)
                
                # Assign the course to the available time slot
                new_tt[day][time_slot] = {
                    "room": available_room,
                    "course": course,
                    "professor": available_professor,
                    "section": section,
                    "batch": section_info["batch"],
                    "department": section_info["department"],
                    "type": courses[course]["type"]
                }
                available_professors[available_professor] = False

    return new_tt


def calc_fit(tt): # FUNCTION TO CALCULATE FITNESS TAKING IN ACCOUNT THE CONSTRAINTS GIVEN IN THE PROJECT DESCRIPTION.
    conflicts = 0
    prof_sched = {}
    for day, slots in tt.items():
        for time_slot, info in slots.items():
            if info:
                professor = info.get("professor")
                if professor:
                    if professor in prof_sched:
                        if day in prof_sched[professor]:
                            conflicts += 1
                        else:
                            prof_sched[professor][day] = [time_slot]
                    else:
                        prof_sched[professor] = {day: [time_slot]}

    sec_sched = {}
    for day, slots in tt.items():
        for time_slot, info in slots.items():
            if info:
                section = info.get("section")
                if section:
                    if section in sec_sched:
                        if day in sec_sched[section]:
                            conflicts += 1
                        else:
                            sec_sched[section][day] = [time_slot]
                    else:
                        sec_sched[section] = {day: [time_slot]}

    room_sched = {}
    for day, slots in tt.items():
        for time_slot, info in slots.items():
            if info:
                room = info.get("room")
                if room:
                    if room in room_sched:
                        if day in room_sched[room]:
                            conflicts += 1
                        else:
                            room_sched[room][day] = [time_slot]
                    else:
                        room_sched[room] = {day: [time_slot]}

    course_sched = {}
    for day, slots in tt.items():
        for time_slot, info in slots.items():
            if info:
                course = info.get("course")
                if course:
                    if course in course_sched:
                        course_days = list(course_sched[course].keys())
                        if day in course_days or (day == "Monday" and "Friday" in course_days) or (
                                day == "Friday" and "Monday" in course_days):
                            conflicts += 1
                        else:
                            course_sched[course][day] = [time_slot]
                    else:
                        course_sched[course] = {day: [time_slot]}

    for day, slots in tt.items():
        for time_slot, info in slots.items():
            if info:
                course = info.get("course")
                if course and courses[course]["type"] == "lab":
                    if (time_slot == "08:30" and "10:00" not in slots) or (
                            time_slot == "13:00" and "14:30" not in slots):
                        conflicts += 1

    return -conflicts


def tour_sel(population, tournament_size): # SELECTION PROCESS, IT BASICALLY USES TOURNAMENT SELECTION
    selected = []
    while len(selected) < tournament_size:
        tournament_candidates = random.sample(population, tournament_size)
        fitness_scores = [calc_fit(tt) for tt in tournament_candidates]
        winner_index = fitness_scores.index(max(fitness_scores))
        selected.append(tournament_candidates[winner_index])
    return selected


def cross(p1, p2): # CROSS OVER 2 POINT (2 POINT CROSSOVER)
    crossover_point = random.choice(list(p1.keys()))
    
    c1 = {}
    c2 = {}
    
    for day in p1:
        if day < crossover_point:
            c1[day] = p1[day]
            c2[day] = p2[day]
        else:
            c1[day] = p2[day]
            c2[day] = p1[day]
    
    return c1, c2


def cross_pop(population): # CROSSOVER POPLUATION RETURN (2 POINT CROSSOVER)
    new_pop = []
    for i in range(0, len(population), 2):
        parent1 = population[i]
        parent2 = population[i + 1] if i + 1 < len(population) else population[i]
        child1, child2 = cross(parent1, parent2)
        new_pop.extend([child1, child2])
    return new_pop


mut_prob = 0.5

def mut_tt(tt): # MUTATION FUNCTION TO APPLY THE MUTATION PROCESS OF THE GENETIC ALGORITHM
    mutated_tt = {}
    for day, slots in tt.items():
        mutated_tt[day] = {}
        for time_slot, info in slots.items():
            if info:
                if random.random() < mut_prob:
                    available_rooms = [room for room in rooms if rooms[room]["type"] == "classroom"]
                    mutated_room = random.choice(available_rooms)
                    info["room"] = mutated_room
                if random.random() < mut_prob:
                    available_professors = [professor for professor in professors if info["course"] in professors[professor]["courses"]]
                    mutated_professor = random.choice(available_professors)
                    info["professor"] = mutated_professor
            mutated_tt[day][time_slot] = info
    return mutated_tt

def mut_pop(population): # RETURNS THE MUTATED POPULATION AFTER APPLYING MUTATION
    mutated_pop = []
    for tt in population:
        mutated_pop.append(mut_tt(tt))
    return mutated_pop


pop_size = 1000 # TAKING 1000 VALUE AS THE INITIAL POPULATION SIZE MEANING GENERATING 1000 POPULATION OF TIMETABLES
init_pop = [generate_tt() for _ in range(pop_size)]

encoded_pop = [encode_timetable(tt) for tt in init_pop] # ENCODING THE INITIAL POPULATION

max_iter = 5000 # ITERATION SIZE AS 5000 MEANING IT WOULD APPLY THE GENETIC ALGORITHM 5000 TIMES ON THE INITIAL POPULATION
best_tt = None  # VARIABLE TO STORE THE BEST POPULATION AFTER APPLYINFG THE GENCTIC ALGORITHM
best_fit = float('inf') # TAKING THE BST FTNESS SIZE AS - (NAGTIVE) INFINITY IN THE START TO COMPARE AGAINST THE FITNESS CALCULATION OF THE TIMETABLES

for it in range(max_iter):  # LOOP TO ITERATE 5000 TIMES
    tournament_size = 5 # INITILAIZING THE TOURNAMENT SIZDE AS 5
    sel_parents = tour_sel(init_pop, tournament_size) # SLECTING PARENTS

    next_gen = cross_pop(sel_parents) # STORING THE NEXT GENERATION AFTER APPLYING 2 POINT CROSSOVER

    mut_next_gen = mut_pop(next_gen) # STORING THE NEXT GENERATION AFTER APPLYING MUTATION

    current_pop = mut_next_gen # AND STORING THE POPULATION AFTER APPLYING ALL FITNESS SELECTION CROSSOVER AND MUTATION

    for tt in current_pop: # THIS LOOP WILL STORE THE BEST TIMETABLE ACCORDING TO FITNESS WHICH IS CLOSER TO 0
        fitness = calc_fit(tt)
        if abs(fitness) < abs(best_fit):
            best_tt = tt
            best_fit = fitness



print("Best Timetable:") # THIS PRINTS THE BEST TIMETABLE AFTER FINDING THE BEST TIMETABLE ACCORDING TO FITNESS
for day, slots in best_tt.items():
    for time_slot, info in slots.items():
        if info:
            print(f"    Day: {day}, Time: {time_slot}, Info: {info}")
        else:
            print(f"    Day: {day}, Time: {time_slot}, Info: EMPTY")
print(f"\nBest Fitness Score: {best_fit}")



Best Timetable:
    Day: Monday, Time: 08:30, Info: {'room': 'C302', 'course': 'CS102', 'professor': 'Prof1', 'section': 'CS-A', 'batch': 20, 'department': 'CS', 'type': 'theory'}
    Day: Monday, Time: 10:00, Info: {'room': 'C305', 'course': 'CS103', 'professor': 'Prof1', 'section': 'CS-A', 'batch': 20, 'department': 'CS', 'type': 'theory'}
    Day: Monday, Time: 11:30, Info: {'room': 'C305', 'course': 'CS104', 'professor': 'Prof1', 'section': 'DS-B', 'batch': 21, 'department': 'DS', 'type': 'theory'}
    Day: Monday, Time: 13:00, Info: {'room': 'C303', 'course': 'CS105', 'professor': 'Prof2', 'section': 'DS-B', 'batch': 21, 'department': 'DS', 'type': 'theory'}
    Day: Monday, Time: 14:30, Info: {'room': 'C302', 'course': 'CS101', 'professor': 'Prof1', 'section': 'CS-A', 'batch': 20, 'department': 'CS', 'type': 'theory'}
    Day: Tuesday, Time: 08:30, Info: {'room': 'C302', 'course': 'CS109', 'professor': 'Prof3', 'section': 'CY-C', 'batch': 22, 'department': 'CY', 'type': 'theory'}